In [1]:
import numpy as np
import statsmodels.api as sm
from scipy.stats import t as t_dist

# 1

In [2]:
def generate_X(n, rho, rng):
    cov = np.array([[1, rho], [rho, 1]])
    X = rng.multivariate_normal(mean=[0, 0], cov=cov, size=n)
    return X[:, 0], X[:, 1]

def generate_Y(X1, X2, beta1, beta2, sigma2, rng):
    e = rng.normal(0, np.sqrt(sigma2), size=len(X1))
    return beta1 * X1 + beta2 * X2 + e

def t_ci(beta_hat, se, df, alpha):
    tcrit = t_dist.ppf(1 - alpha/2, df)
    return beta_hat - tcrit * se, beta_hat + tcrit * se

def two_sided_t_pvalue(t_stat, df):
    return 2 * (1 - t_dist.cdf(abs(t_stat), df))

def fit_ols(X, y):
    return sm.OLS(y, X).fit()

def algorithm_1(X1, X2, y, alpha):
    n = len(y)

    # model S: y ~ X1
    XS = X1.reshape(n, 1)
    resS = fit_ols(XS, y)
    beta1_S = resS.params[0]
    se1_S = resS.bse[0]
    df_S = resS.df_resid
    ci1_S = t_ci(beta1_S, se1_S, df_S, alpha)

    # model 3: y ~ X1 + X2
    X3 = np.column_stack([X1, X2])
    res3 = fit_ols(X3, y)
    beta1_3 = res3.params[0]
    se1_3 = res3.bse[0]
    beta2_3 = res3.params[1]
    se2_3 = res3.bse[1]
    df_3 = res3.df_resid

    t_b2 = beta2_3 / se2_3
    p_b2 = two_sided_t_pvalue(t_b2, df_3)
    ci1_3 = t_ci(beta1_3, se1_3, df_3, alpha)

    if p_b2 < alpha:
        # select full model
        return beta1_3, ci1_3, "3", p_b2
    else:
        # select reduced model
        return beta1_S, ci1_S, "S", p_b2
    
def run_one_replication(n, rho, beta1, beta2, sigma2, alpha):
    rng = np.random.default_rng(0)
    X1, X2 = generate_X(n, rho, rng)
    y = generate_Y(X1, X2, beta1, beta2, sigma2, rng)
    return algorithm_1(X1, X2, y, alpha)

In [ ]:
N = 20000
n = 60
beta1 = 1
sigma2 = 0.25
alpha = 0.05
rhos = np.array([0, 0.2, 0.4, 0.6, 0.8, 0.99])
beta2s = np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2])

## a